## Mortgage Data
Annabella Stoll-Dansereau

#### Data Source 
In this section I read in all the mortgage data for select counties in California, Arizona, and Nevada derived from Querying a series of microsoft access databases. This data was sourced from FHMA and we are using 2012-2019 (since the data format changes post 2019 and I was worried about the consistency). 

#### Description of the following code 

These imports include Sacramento, Fresno, Las Vegas, Reno, Tuscan, and Phoenix as these seemed like the most similar cities in Arizona. Later on I subset it to just Fresno as my group to create a synetheic control hence why I only am normalizing this county data. Essentially it is ~5% cheaper to live in fresno from real estate value and similar percentage different in the county's average incomes. My covariates here are low income mortgage counts, income tract ratio (median income in this tract / tract area median income), and percentage minority in this tract. Initially I wanted to add in census data for additional covariates such as home ownership rates however due to computing restrictions this didnt seem appropriate for this projects timeline.

In [4]:
import pandas as pd 
#run once and then import csv since it take ~ hour
years = [2012, 2013]#, 2014, 2015, 2016, 2017, 2018, 2019]
collapsed_df = None

def count_low_income_mortgages(x):
    return (x < 98500).sum()

for n, year in enumerate(years):
    df = pd.read_excel(rf"C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\State County {year}.xlsx")

    if n == 0:
        # Initial year, establish the collapsed_df with additional data
        collapsed_df = df.groupby(['Census Tract - 2010 Census']).agg({
            'US Postal State Code': 'first',
            'County - 2010 Census': 'first',
            '2010 Census Tract - Percent Minority': 'first',
            'Tract Income Ratio': 'first',
            #'Property Type': 'size',
            'Borrower(s) Annual Income': count_low_income_mortgages,
        }).reset_index()
        collapsed_df['Tract Income Ratio'] = collapsed_df['Tract Income Ratio'] * 100
        collapsed_df = collapsed_df.rename(columns={'Borrower(s) Annual Income': f'Num Low Income Mortgages {year}'})
    else:
        # Subsequent years, only get the count data
        temp_df = df.groupby(['Census Tract - 2010 Census']).agg({
            #'Property Type': 'size',
            'Borrower(s) Annual Income': count_low_income_mortgages,
        }).reset_index()
        temp_df = temp_df.rename(columns={'Borrower(s) Annual Income': f'Num Low Income Mortgages {year}'})
        
        # Merge with the existing collapsed_df
        if collapsed_df is not None:
            collapsed_df = collapsed_df.merge(temp_df, on='Census Tract - 2010 Census', how='left', suffixes=('', f' {year}'))
            collapsed_df = collapsed_df.fillna(0)
    print(f'done {year}')

#collapsed_df.to_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\20122019_collapsed_mortgage.csv', index=False)
       

done 2012
done 2013


In [6]:
collapsed_df
x = pd.read_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\20122019_collapsed_mortgage.csv')
x

,Census Tract - 2010 Census,US Postal State Code,County - 2010 Census,2010 Census Tract - Percent Minority,Tract Income Ratio,Num Low Income Mortgages 2012,Num Low Income Mortgages 2013,Num Low Income Mortgages 2014,Num Low Income Mortgages 2015,Num Low Income Mortgages 2016,Num Low Income Mortgages 2017,Num Low Income Mortgages 2018,Num Low Income Mortgages 2019
0,0,4,21,9999.000000,999900.000000,13,10.0,1.0,0.0,0.0,0.0,1.0,0.0
1,100,6,67,17.341203,143.195110,116,97.0,51.0,57.0,50.0,43.0,29.0,52.0
2,101,32,3,73.142607,82.877849,60,72.0,37.0,58.0,65.0,74.0,110.0,98.0
3,102,32,31,34.347275,46.479584,5,14.0,8.0,9.0,2.0,10.0,4.0,2.0
4,103,32,3,72.790080,63.025920,21,28.0,22.0,23.0,31.0,32.0,39.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298,960303,32,19,17.185148,106.384534,114,101.0,63.0,92.0,79.0,105.0,94.0,106.0
2299,960800,32,19,23.752446,82.756315,41,61.0,27.0,24.0,32.0,31.0,23.0,28.0
2300,960900,32,19,30.047549,69.383787,40,66.0,37.0,25.0,30.0,42.0,37.0,41.0
2301,970700,32,21,22.387191,103.504106,6,15.0,10.0,7.0,10.0,7.0,9.0,14.0


In [9]:
import pandas as pd 

years = [2012, 2013] # Shortened for example
collapsed_df = None

# Thresholds dictionary
thresholds = {
    3: 98500, #Las Vegas county
    50: 107756, #Fresno county
    67: 119729, #Sacramento 
    31: 104637, #Reno
    19: 104336, #Tucson
    13: 103531, #Phoenix
    21: 103531 #Phoenix

}

# Function for counting low income mortgages
def count_low_income_mortgages(x, county_code):
    threshold = thresholds.get(county_code, 98500) # Default threshold
    return (x < threshold).sum()

for n, year in enumerate(years):
    df = pd.read_excel(rf"C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\State County {year}.xlsx")
    #this is just to make life more simple since we have two of the same counties in the same state 
    df.loc[(df['US Postal State Code'] == 6) & (df['County - 2010 Census'] == 19), 'County - 2010 Census'] = 50

    if n == 0:
        # Initial year, establish the collapsed_df with additional data
        collapsed_df = df.groupby(['Census Tract - 2010 Census']).apply(
            lambda group: pd.Series({
                'US Postal State Code': group['US Postal State Code'].iloc[0],
                'County - 2010 Census': group['County - 2010 Census'].iloc[0],
                '2010 Census Tract - Percent Minority': group['2010 Census Tract - Percent Minority'].iloc[0],
                'Tract Income Ratio': group['Tract Income Ratio'].iloc[0] * 100,
                f'Num Low Income Mortgages {year}': count_low_income_mortgages(group['Borrower(s) Annual Income'], group['County - 2010 Census'].iloc[0])
            })
        ).reset_index()
        
    else:
        # Subsequent years, only get the count data
        temp_df = df.groupby(['Census Tract - 2010 Census']).apply(
            lambda group: pd.Series({
                f'Num Low Income Mortgages {year}': count_low_income_mortgages(group['Borrower(s) Annual Income'], group['County - 2010 Census'].iloc[0])
            })
        ).reset_index()

        # Merge with the existing collapsed_df
        collapsed_df = collapsed_df.merge(temp_df, on='Census Tract - 2010 Census', how='left')
        collapsed_df = collapsed_df.fillna(0)

    print(f'done {year}')

# Save to CSV
# collapsed_df.to_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\20122019_collapsed_mortgage.csv', index=False)


      Census Tract - 2010 Census  US Postal State Code  County - 2010 Census  \
0                              0                   4.0                  21.0   
1                            100                   6.0                  67.0   
2                            101                  32.0                   3.0   
3                            102                  32.0                  31.0   
4                            103                  32.0                   3.0   
...                          ...                   ...                   ...   
2298                      960303                  32.0                  19.0   
2299                      960800                  32.0                  19.0   
2300                      960900                  32.0                  19.0   
2301                      970700                  32.0                  21.0   
2302                      980000                  32.0                  31.0   

      2010 Census Tract - Percent Minor

#### Matching

Here I am creating the synthetic controls. Following the matching strategy used by "CITE TEXTBOOK" I created a synthetical control based on my covariates from 2012 to 2017. To slightly speed up the algorythim instead of starting at uniform weights I had it start at the weights found for the first iteration. This is from the guess that many of the treated tracts in Las Vegas are simimilar

In [17]:
import pandas as pd 
from scipy.optimize import fmin_slsqp
from toolz import partial
import numpy as np 
import matplotlib.pyplot as plt

collapsed_df = pd.read_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\20122019_collapsed_mortgage.csv')
crosswalk = pd.read_csv(r"C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\filtered_zip_tract.csv")
crosswalk.columns = crosswalk.columns.str.lstrip('0')

def loss_w(W, X, y) -> float:
    return np.sqrt(np.mean((y - X.dot(W))**2))

def get_w(X, y, initial_weights=None):
    if initial_weights is None:
        initial_weights = [1/X.shape[1]] * X.shape[1]
    
    weights = fmin_slsqp(partial(loss_w, X=X, y=y),
                         np.array(initial_weights),
                         f_eqcons=lambda x: np.sum(x) - 1,
                         bounds=[(0.0, 1.0)] * len(initial_weights),
                         disp=False)
    return weights

try: #just for when I rerun so it doesnt error
    #For the control I am just using Fresno and Tuscan as it takes too long to run with all the other cities 
    control_pool = collapsed_df[(collapsed_df['County - 2010 Census'] == 19) | (collapsed_df['County - 2010 Census'] == 50)].drop(columns=['US Postal State Code', 'County - 2010 Census']) 
    treated = collapsed_df[collapsed_df['Census Tract - 2010 Census'].isin(crosswalk['TRACTCE'])]
    treated = treated[treated['County - 2010 Census'] == 3].drop(columns=['US Postal State Code', 'County - 2010 Census']) #some overlap between county and codes so make sure its the correct filter
    print("Length of treated:", len(treated))
    print("Length of control_pool:", len(control_pool))
except:
    pass

treated = treated.melt(id_vars=['Census Tract - 2010 Census'], var_name='variable', value_name='value')
treated = treated.pivot(index='variable', columns='Census Tract - 2010 Census', values='value')
control_pool = control_pool.melt(id_vars=['Census Tract - 2010 Census'], var_name='variable', value_name='value')
control_pool = control_pool.pivot(index='variable', columns='Census Tract - 2010 Census', values='value')
#control_pool=control_pool.iloc[:, :10] for if you want to test and have it work in finite time
synthetic_controls = {}

#dont want to train on what we want to predict after the policy occurs
rows_to_keep = ~treated.index.str.contains('2018|2019')
treated_dropped = treated[rows_to_keep]
control_dropped = control_pool[rows_to_keep]
synthetic_controls_df = pd.DataFrame()

first_iteration = True
stored_weights = None

#for n, column in enumerate(treated.columns):
for n, column in enumerate(treated_dropped.columns[:2]):
    print(n)
    y = treated_dropped[column].values  # Treated unit's values
    X = control_dropped.values          # Control units' values
    #slightly faster to start here rather than uniform
    if first_iteration:
        weights = get_w(X, y)
        stored_weights = weights
        first_iteration = False
    else:
        weights = get_w(X, y, initial_weights=stored_weights)

    synthetic_control = control_pool.dot(weights)
    # Store the synthetic control series in the dictionary more efficient than df
    synthetic_controls[f'syn_{column}'] = synthetic_control
synthetic_controls_df = pd.DataFrame(synthetic_controls)
treated = pd.concat([treated, synthetic_controls_df], axis=1)

#save as a treated csv
#treated.to_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\treated_dataframe.csv', index=True)

Length of treated: 236
Length of control_pool: 335
0
1


In [23]:
treated = pd.read_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\treated_dataframe.csv')
#this is getting rid of the Tract Income Ratio which is the last column and 2010 Census Tract - Percent Minority which is first
treated = treated.drop([0, len(treated)-1]) 

,101,103,105,106,107,108,109,201,203,301,...,syn_5437,syn_5438,syn_5439,syn_5613,syn_5902,syn_6001,syn_6104,syn_6900,syn_7100,syn_7200
1,60.0,21.0,27.0,53.0,27.0,17.0,23.0,293.0,45.0,26.0,...,55.920386,8.788383,32.571464,52.999985,7.353962,7.187161,53.004749,82.906773,35.400545,38.000232
2,72.0,28.0,42.0,39.0,27.0,18.0,31.0,223.0,61.0,44.0,...,52.389493,13.796025,42.325343,42.999996,12.309791,15.974839,54.998286,89.159195,34.504025,43.999980
3,37.0,22.0,21.0,23.0,14.0,9.0,21.0,99.0,39.0,23.0,...,42.960454,9.701262,18.885985,28.000009,7.410910,9.209835,43.992300,65.148858,26.106956,19.000297
4,58.0,23.0,28.0,24.0,12.0,10.0,14.0,158.0,27.0,16.0,...,41.283573,7.355799,24.976718,30.999992,5.528676,7.367002,40.019789,69.565706,23.126676,29.000386
5,65.0,31.0,44.0,27.0,17.0,13.0,16.0,152.0,37.0,21.0,...,38.004091,6.778881,31.371907,36.999988,5.349901,4.740816,64.986500,75.348983,38.141817,34.999441
6,74.0,32.0,37.0,35.0,27.0,20.0,36.0,165.0,39.0,24.0,...,48.611460,8.942568,48.654436,33.000027,6.340296,10.608666,61.001018,75.469593,40.327846,46.000376
7,110.0,39.0,30.0,48.0,31.0,32.0,31.0,156.0,39.0,35.0,...,38.309127,10.979931,47.928047,33.967488,7.896469,9.974434,69.701340,74.671880,41.341108,40.621363
8,98.0,35.0,34.0,60.0,19.0,25.0,46.0,277.0,45.0,49.0,...,48.890097,13.213548,52.025020,31.547907,9.908651,12.924343,67.472160,72.167984,44.680523,41.177969


In [25]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Read out treated df from above since it takes forever to run
treated = pd.read_csv(r'C:\Users\annab\OneDrive\Desktop\Year 4 fun stuff\526\morgage data\treated_dataframe.csv')
#this is getting rid of the Tract Income Ratio which is the last column and 2010 Census Tract - Percent Minority which is first
treated = treated.drop([0, len(treated)-1]) 
#get a list of all the treated tracts
tracts = treated.columns[:len(treated.columns)//2].tolist()
all_tracts_long_df = pd.DataFrame()

# Loop over each tract to transpose and melt the data into a format we can run the DiD on
for tract in tracts:
    col_df = treated[[tract, f'syn_{tract}']].transpose()
    
    # Rename the columns to years observed
    new_column_names = {i: year for i, year in enumerate(range(2012, 2012 + len(col_df.columns)), start=1)}
    col_df = col_df.rename(columns=new_column_names)
    long_df = col_df.reset_index().melt(id_vars='index', var_name='Year', value_name='Value')
    
    long_df = long_df.rename(columns={'index': 'Group'})
    long_df['Tract'] = tract  # keep track of the tract
    # Append to the all_tracts_long_df DataFrame
    all_tracts_long_df = pd.concat([all_tracts_long_df, long_df], ignore_index=True)

all_tracts_long_df['Year'] = all_tracts_long_df['Year'].astype(int)
# Create 'treated' and 'post' columns
all_tracts_long_df['treated'] = all_tracts_long_df['Group'].apply(lambda x: 1 if 'syn' not in x else 0)
all_tracts_long_df['post'] = all_tracts_long_df['Year'].apply(lambda x: 1 if x >= 2018 else 0)

# Fit the DiD model with 'Tract' as a fixed effect
did_formula = 'Value ~ treated + post + treated:post + C(Tract)'
did_model = smf.ols(formula=did_formula, data=all_tracts_long_df).fit()
print(did_model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     97.37
Date:                Sat, 02 Dec 2023   Prob (F-statistic):               0.00
Time:                        16:19:21   Log-Likelihood:                -14979.
No. Observations:                3776   AIC:                         3.044e+04
Df Residuals:                    3537   BIC:                         3.193e+04
Df Model:                         238                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           75.9033      3.313  